In [19]:
import tensorflow as tf
from attrdict import AttrDict
tf.__version__

'1.8.0'

读取训练和测试文件

In [20]:
import re
import tarfile

class ReviewReader:
    
    TOKEN_REGEX = re.compile(r'[A-Za-z]+|[!?.:,()]')
    
    def __init__(self):
        filepath="aclImdb_v1.tar.gz"
        self.train_data = []
        self.test_data = []
        
        with tarfile.open(filepath) as archive:
            for filename in archive.getnames():
                if filename.startswith('aclImdb/train/pos/'):
                    self.train_data.append((self._read(archive, filename), True))
                elif filename.startswith('aclImdb/train/neg/'):
                    self.train_data.append((self._read(archive, filename), False))
                elif filename.startswith('aclImdb/test/pos/'):
                    self.test_data.append((self._read(archive, filename), True))
                elif filename.startswith('aclImdb/test/neg/'):
                    self.test_data.append((self._read(archive, filename), False))
    
    def _read(self, archive, filename):
        with archive.extractfile(filename) as file_:
            data = file_.read().decode('utf-8')
            data = type(self).TOKEN_REGEX.findall(data)
            data = [x.lower() for x in data]
            return data
    
    

    

In [17]:
rr = ReviewReader()
test_data = rr.test_data
train_data = rr.train_data
print("test.shape:", len(test_data))
print("train.shape:", len(train_data))

test.shape: 25000
train.shape: 25000


读取embedding

In [25]:
from load_word2vec import load_embedding

class Embedding:
    def __init__(self, vocab_path="", embedding_path= "", length=0):
        vocab, index_vocab, embed = load_embedding()
        self._length = length
        self._embedding = embed
        self._vocab = vocab

    def __call__(self, sequence):
        data = np.zeros((self._length, self._embedding.shape[1]))
        indices = [self._vocab.get(x, 0) for x in sequence]

        logging.info("indices:", indices)
        embedded = self._embedding[indices, :]
        data[:len(sequence)] = embedded
        return data

    @property
    def dimensions(self):
        return self._embedding.shape[1]

In [ ]:
定义运算的图


In [31]:
main_graph = tf.Graph()
# embed = Embedding()

length = 2700  # 每个句子的最大长度
embedding_dimensions = 300   # 每个word embedding的维数

data_input = tf.placeholder(tf.float32, [None, length, embedding_dimensions], name="data")
data_label = tf.placeholder(tf.float32, [None, 2], name = "label")

hidden_size = 150
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size)



output, state = tf.nn.dynamic_rnn(
    rnn_cell,
    data_input,
    dtype = tf.float32,
    
)

session = tf.Session()
session.run(output, feed_dict={data_input:train_data[:20]})







ValueError: Variable rnn/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-29-45b82bab828a>", line 18, in <module>
    dtype = tf.float32,
  File "/Users/liujingwei/pythonenv/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/liujingwei/pythonenv/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2903, in run_ast_nodes
    if self.run_code(code, result):


In [28]:


length = 2700
embedding_dimensions = 300

params = AttrDict(
        rnn_cell = tf.nn.rnn_cell.GRUCell,
        rnn_hidden = 300,
        optimizer = tf.train.RMSPropOptimizer(0.002),
        batch_size = 20,
    )

with graph.as_default():
    data = tf.placeholder(tf.float32, [None, length, embedding_dimensions], name="data")
    target = tf.placeholder(tf.float32, [None, 2], name = "target")
    used = tf.sign(tf.reduce_max(tf.abs(data), reduction_indices = 2))
    temp_length = tf.reduce_sum(used, reduction_indices = 1)
    seq_length = tf.cast(temp_length, tf.int32)

    output, _ = tf.nn.dynamic_rnn(params.rnn_cell(params.rnn_hidden),
        data,
        dtype = tf.float32,
        sequence_length = seq_length
    )
    

In [29]:
writer = tf.summary.FileWriter("./graph_temp", graph)
writer.close()